In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from IPython import display

import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import pickle
import math
import re

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec, Word2Vec
import logging

from word2vec_as_MF import Word2vecMF
from functions import *

import time
dimension = 20
model =  Word2vecMF()